In [ ]:
import pandas as pd
import plotly.express as px, plotly.graph_objects as go


In [ ]:
# Load the dataset
file_path = 'Openrice_9006_restaurants_cleaned_v4.csv'
df = pd.read_csv(file_path, encoding='UTF-8-SIG')

# Define the categories with exact column names
categories = ['comment_taste_star', 'comment_env_star', 'comment_service_star', 
              'comment_hygiene_star', 'comment_fair_price_star']
display_names = ['Taste', 'Environment', 'Service', 'Hygiene', 'Fairness']
ratings = [1.0, 2.0, 3.0, 4.0, 5.0]

# Define reversed ratings and colors first
reversed_ratings = ratings[::-1]  # This creates [5.0, 4.0, 3.0, 2.0, 1.0]
colors = ['#FF9999', '#DAA520', '#32CD32', '#20B2AA', '#DDA0DD']
reversed_colors = colors[::-1]

# Function to calculate percentages
def calculate_percentages(df, category):
    total = len(df[category].dropna())
    percentages = []
    for rating in ratings:
        count = len(df[df[category] == rating])
        percentage = (count / total) * 100 if total > 0 else 0
        percentages.append(percentage)
    return percentages

# Calculate the average rating for each category
category_averages = {cat: df[cat].mean() for cat in categories}

# Sort categories by their average rating in descending order
sorted_categories = sorted(category_averages, key=category_averages.get, reverse=True) # type: ignore
sorted_display_names = [display_names[categories.index(cat)] for cat in sorted_categories]

# Create the figure
fig = go.Figure()

# Add bars for each rating in reversed order
for i, rating in enumerate(reversed_ratings):
    percentages = [calculate_percentages(df, cat)[ratings.index(rating)] for cat in sorted_categories]
    fig.add_trace(go.Bar(
        name=str(rating),
        x=sorted_display_names,
        y=percentages,
        marker_color=reversed_colors[i]
    ))

# Update layout
fig.update_layout(
    barmode='stack',
    title='Distribution of Comment Star Percentages by Rating Level',
    yaxis_title='Percentage',
    xaxis_title='Categories',
    showlegend=True,
    legend_title='Ratings',
    template='plotly_white'
)

# Update y-axis
fig.update_yaxes(range=[0, 100])

# Display the figure
fig.show()


In [2]:
# Define main cuisine categories
main_categories = {
    '港式': ['港式', '茶餐廳', '冰室', '大排檔'],
    '火鍋': ['火鍋', '打邊爐'],
    '中菜': ['中菜', '中式', '粵菜', '廣東菜', '粵菜', '廣東', '川菜', '四川', '北京菜', '京菜', '上海菜', '滬菜'],
    '日本菜': ['日本', '日式', '壽司', '刺身', '拉麵', '壽喜燒'],
    '韓國菜': ['韓國', '韓式', '韓國菜',],
    '越南菜': ['越式', '越南', '越南菜'],
    '泰國菜': ['泰國', '泰式', '泰國菜'],
    '台灣菜': ['台灣', '台式', '台灣菜'],
    '西式': ['西式', '西餐', '英國菜', '美國菜', '意大利', '意式', '意大利菜', '西班牙', '西班牙菜', '葡國', '葡國菜', '葡萄牙', '葡萄牙菜'],
    '法國菜': ['法國', '法式', '法國菜'],
    '甜品': ['甜品', '甜點', '甜品店', '甜品專門店', '糖水', '糖水店', '糖水鋪', '甜品鋪', '糖水舖', '甜品舖'],
    '咖啡': ['咖啡', '咖啡店', '咖啡室', '咖啡館', '咖啡屋', '咖啡廳', '咖啡專門店'],
    '任食/放題': ['任食', '放題', '任食放題', '任食任點'],
    '自助餐': ['自助餐', '自助餐廳', '自助餐館', '自助餐飲', '自助', 'buffet', 'Buffet', 'BUFFET'],
    '素食': ['素食', '齋', '蔬食', '素菜', '素', '純素', '素食主義'], 
}

# Function to categorize cuisine types
def categorize_cuisine(cuisine):
    if pd.isna(cuisine):
        return 'Other'
    for main_cat, sub_cats in main_categories.items():
        if any(sub_cat in str(cuisine) for sub_cat in sub_cats):
            return main_cat
    return 'Other'


In [4]:
#alternative 

# Ensure the dataset has no duplicate restaurants
df = df.drop_duplicates(subset='restaurant_name')

# Categorize cuisine types using predefined categories
def categorize_cuisine(cuisine):
    if pd.isna(cuisine):
        return 'Other'
    for category, keywords in main_categories.items():
        if any(keyword in cuisine for keyword in keywords):
            return category
    return 'Other'

df['cleaned_cuisine_type'] = df['restaurant_cuisine_type'].apply(categorize_cuisine)

# Group by cuisine type, count distinct restaurants, and sort descending
cuisine_counts = (
    df.groupby('cleaned_cuisine_type')['restaurant_name']
    .nunique()
    .reset_index(name='distinct_restaurant_count')
    .sort_values(by='distinct_restaurant_count', ascending=False)
)

# Remove 'Other' category for clarity
cuisine_counts = cuisine_counts[cuisine_counts['cleaned_cuisine_type'] != 'Other']

# Define colors for visualization
# Define colors dynamically
color_map = {
    '港式': '#FF6B6B', '火鍋': '#F17F29', '中菜': '#96CEB4', '日本菜': '#4ECDC4',
    '韓國菜': '#851E3E', '越南菜': '#FF9F1C', '泰國菜': '#FFD93D', '台灣菜': '#FFEEAD',
    '西式': '#45B7D1', '法國菜': '#6C5B7B', '甜品': '#FFD93D', '咖啡': '#A0522D',
    '任食/放題': '#FF6F61', '自助餐': '#FF7F50', '素食': '#7AC74F'
}

# Assign colors dynamically based on sorted cuisine types
sorted_colors = [color_map.get(cuisine, '#D3D3D3') for cuisine in cuisine_counts['cleaned_cuisine_type']]

# Calculate percentages for labels
cuisine_counts['percentage'] = (cuisine_counts['distinct_restaurant_count'] / 
                                cuisine_counts['distinct_restaurant_count'].sum()) * 100

# Add a trace for a doughnut chart
fig = go.Figure()
fig.add_trace(go.Pie(
    labels=cuisine_counts['cleaned_cuisine_type'],
    values=cuisine_counts['distinct_restaurant_count'],
    marker_colors=sorted_colors,
    textinfo='label+percent',
    hoverinfo='label+value+percent',
    hole=0.4  # Creates the doughnut effect
))

# Update layout for a more modern and fancy look
fig.update_layout(
    title='Distribution of Distinct Restaurants by Cuisine Types',
    template='plotly_white',
    height=700,
    width=800,
    legend=dict(
        title="Cuisine Types",
        orientation="v",
        yanchor="middle",
        y=0.5,
        xanchor="right",
        x=1.2
    ),
    annotations=[
        dict(
            text="Cuisines",
            x=0.5,
            y=0.5,
            font_size=20,
            showarrow=False
        )
    ]
)

fig.show()


In [ ]:
# Read and clean the data
df['comment_price_per_person'] = pd.to_numeric(df['comment_price_per_person'], errors='coerce')
df = df[df['comment_price_per_person'] < 10000]

# Apply categorization
df['cuisine_category'] = df['restaurant_cuisine_type'].apply(categorize_cuisine)

# Create interactive boxplot with plotly
fig = px.box(df, 
             x='cuisine_category', 
             y='comment_price_per_person',
             title='Price Distribution by Cuisine Type',
             labels={'cuisine_category': 'Cuisine Type',
                    'comment_price_per_person': 'Price per Person (HKD)'},
             height=600,
             color='cuisine_category',
             color_discrete_sequence=px.colors.qualitative.Set3)

# Update layout for a fancier look
fig.update_layout(
    showlegend=False,
    xaxis_tickangle=45,
    margin=dict(t=100, l=50, r=50, b=100),
    plot_bgcolor='rgba(240,240,240,0.95)',
    paper_bgcolor='rgba(255,255,255,1)',
    font=dict(size=12, color='black')
)

# Add gridlines and hover effects
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Show the plot
fig.show()


In [7]:
# Apply categorization to create the 'cuisine_category' column
df['cuisine_category'] = df['restaurant_cuisine_type'].apply(categorize_cuisine)

# Display the first few rows to verify the new column
df.head()


,Column1,scraping_status,restaurant_name,restaurant_link,restaurant_comment_page_link,restaurant_overall_rating,restaurant_cuisine_type,restaurant_price_range,restaurant_smiley_smile,restaurant_smiley_ok,...,comment_service_star,comment_hygiene_star,comment_fair_price_star,comment_dining_date,comment_dining_time,comment_dining_way,comment_price_per_person,comment_recommended_dishes,comment_text_with_emoji,cuisine_category
0,0,"Failed, need to re-scrape the page",NaN,https://www.openrice.com//zh/hongkong/r-taylor...,https://www.openrice.com//zh/hongkong/r-taylor...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
1,1,Success,Taylor Shellfish Farms (高街),https://www.openrice.com//zh/hongkong/r-taylor...,https://www.openrice.com//zh/hongkong/r-taylor...,4.5,多國菜海鮮蠔吧,$201-400,109.0,12.0,...,5.0,5.0,4.0,3/6/2023 0:00,NaN,堂食,NaN,生蠔\nSeafood Stew,坐咗喺户外區，雖然今日3幾度，但有部大風扇吹住，感覺都ok，貪佢户外有幅植物牆，又人少少，都...,Other
2,2,Success,Taylor Shellfish Farms (高街),https://www.openrice.com//zh/hongkong/r-taylor...,https://www.openrice.com//zh/hongkong/r-taylor...,4.5,多國菜海鮮蠔吧,$201-400,109.0,12.0,...,5.0,5.0,4.0,3/6/2023 0:00,晚餐,堂食,300.0,Seafood Stew,今日朋友生日，第一次book位試呢間海鮮餐廳，一嚟到發現侍應貼心咁咗門口露天同餐廳最入較靜位...,Other
3,3,Success,Taylor Shellfish Farms (高街),https://www.openrice.com//zh/hongkong/r-taylor...,https://www.openrice.com//zh/hongkong/r-taylor...,4.5,多國菜海鮮蠔吧,$201-400,109.0,12.0,...,5.0,5.0,4.0,3/6/2023 0:00,晚餐,堂食,400.0,Oysters\nSeafood Stew,見到呢間餐廳已經一段時間，有次睇到介紹原來呢度嘅蠔都出名～仲有自家製嘅蠔場，所以訂咗位嚟試吓...,Other
4,4,Success,Taylor Shellfish Farms (高街),https://www.openrice.com//zh/hongkong/r-taylor...,https://www.openrice.com//zh/hongkong/r-taylor...,4.5,多國菜海鮮蠔吧,$201-400,109.0,12.0,...,5.0,4.0,2.0,NaN,NaN,堂食,NaN,NaN,嚟呢度食慶生飯，完全係睇網上介紹惹來嘅，位置到有少少難去，不過彩都搵到。因為大家都鍾意食海鮮...,Other


In [ ]:
# Define main cuisine categories
main_categories = {
    '港式': ['港式', '茶餐廳', '冰室', '大排檔'],
    '火鍋': ['火鍋', '打邊爐'],
    '中菜': ['中菜', '中式', '粵菜', '廣東菜', '粵菜', '廣東', '川菜', '四川', '北京菜', '京菜', '上海菜', '滬菜'],
    '日本菜': ['日本', '日式', '壽司', '刺身', '拉麵', '壽喜燒'],
    '韓國菜': ['韓國', '韓式', '韓國菜',],
    '越南菜': ['越式', '越南', '越南菜'],
    '泰國菜': ['泰國', '泰式', '泰國菜'],
    '台灣菜': ['台灣', '台式', '台灣菜'],
    '西式': ['西式', '西餐', '英國菜', '美國菜', '意大利', '意式', '意大利菜', '西班牙', '西班牙菜', '葡國', '葡國菜', '葡萄牙', '葡萄牙菜'],
    '法國菜': ['法國', '法式', '法國菜'],
    '甜品': ['甜品', '甜點', '甜品店', '甜品專門店', '糖水', '糖水店', '糖水鋪', '甜品鋪', '糖水舖', '甜品舖'],
    '咖啡': ['咖啡', '咖啡店', '咖啡室', '咖啡館', '咖啡屋', '咖啡廳', '咖啡專門店'],
    '任食/放題': ['任食', '放題', '任食放題', '任食任點'],
    '自助餐': ['自助餐', '自助餐廳', '自助餐館', '自助餐飲', '自助', 'buffet', 'Buffet', 'BUFFET'],
    '素食': ['素食', '齋', '蔬食', '素菜', '素', '純素', '素食主義'],
}

def categorize_cuisine(cuisine):
    if pd.isna(cuisine):
        return 'Other'
    for main_cat, sub_cats in main_categories.items():
        if any(sub_cat in str(cuisine) for sub_cat in sub_cats):
            return main_cat
    return 'Other'

# Apply categorization
df['main_cuisine_category'] = df['restaurant_cuisine_type'].apply(categorize_cuisine)

# Remove duplicates
df_unique = df.drop_duplicates(['restaurant_name', 'restaurant_smiley_smile', 'restaurant_smiley_ok', 'restaurant_smiley_cry'])

# Calculate reactions and sort
cuisine_reactions = df_unique.groupby('main_cuisine_category').agg({
    'restaurant_smiley_smile': 'sum',
    'restaurant_smiley_ok': 'sum',
    'restaurant_smiley_cry': 'sum'
}).reset_index()

# Calculate total reactions
cuisine_reactions['Total_Reactions'] = cuisine_reactions[['restaurant_smiley_smile', 'restaurant_smiley_ok', 'restaurant_smiley_cry']].sum(axis=1)
cuisine_reactions = cuisine_reactions.sort_values('Total_Reactions', ascending=False)

# Melt for plotting
cuisine_reactions_melted = cuisine_reactions.melt(
    id_vars=['main_cuisine_category'],
    value_vars=['restaurant_smiley_smile', 'restaurant_smiley_ok', 'restaurant_smiley_cry'],
    var_name='Reaction Type',
    value_name='Total Count'
)

# Create plot
fig = px.bar(cuisine_reactions_melted, 
             x='main_cuisine_category', 
             y='Total Count',
             color='Reaction Type',
             title='Total Smiley Reactions by Cuisine Category ',
             template='plotly_white',
             barmode='stack',
             color_discrete_sequence=['#3CB371', '#FFD700', '#FF6B6B'])

# Update layout
fig.update_layout(
    xaxis_title=None,
    yaxis_title="Number of Reactions",
    title_x=0.5,
    xaxis={'tickangle': 45},
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    height=700,
    width=1200,
    margin=dict(b=150)
)

fig.show()

# Print sorted data
print("\
Sorted reaction counts by cuisine category:")
print(cuisine_reactions[['main_cuisine_category', 'Total_Reactions']])


Sorted reaction counts by cuisine category:
   main_cuisine_category  Total_Reactions
12                    西式         226351.0
4                    日本菜         190250.0
7                     港式         165213.0
1                     中菜          80041.0
0                  Other          78144.0
14                   韓國菜          52723.0
6                    泰國菜          52678.0
8                     火鍋          40599.0
2                    台灣菜          35711.0
3                     咖啡          17568.0
13                   越南菜          13673.0
11                   自助餐           9913.0
5                    法國菜           6879.0
10                    素食           4334.0
9                     甜品            380.0


In [ ]:
# Calculate percentages for each smiley type
smiley_by_cuisine['smile_percentage'] = (smiley_by_cuisine['restaurant_smiley_smile'] / smiley_by_cuisine['total']) * 100
smiley_by_cuisine['ok_percentage'] = (smiley_by_cuisine['restaurant_smiley_ok'] / smiley_by_cuisine['total']) * 100
smiley_by_cuisine['cry_percentage'] = (smiley_by_cuisine['restaurant_smiley_cry'] / smiley_by_cuisine['total']) * 100

# Create stacked bar chart with percentages
fig = go.Figure()

# Add bars for each smiley type
fig.add_trace(go.Bar(
    name='😊 Smile',
    x=smiley_by_cuisine['cuisine_category'],
    y=smiley_by_cuisine['smile_percentage'],
    marker_color='#2ecc71'
))

fig.add_trace(go.Bar(
    name='😐 OK',
    x=smiley_by_cuisine['cuisine_category'],
    y=smiley_by_cuisine['ok_percentage'],
    marker_color='#f1c40f'
))

fig.add_trace(go.Bar(
    name='😢 Cry',
    x=smiley_by_cuisine['cuisine_category'],
    y=smiley_by_cuisine['cry_percentage'],
    marker_color='#e74c3c'
))

# Update layout
fig.update_layout(
    barmode='stack',
    title='Distribution of Smiley Percentages by Cuisine Category',
    xaxis_title='Cuisine Type',
    yaxis_title='Percentage of Smileys',
    template='plotly_white',
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.02
    ),
    margin=dict(r=150)  # Add right margin for legend
)

# Update y-axis range
fig.update_yaxes(range=[0, 100])

# Display the figure
fig.show()


In [ ]:
# Create a grade distribution for each cuisine type
grade_dist = pd.crosstab(df['cuisine_category'], df['comment_account_grade'])

# Sort by total reviews
grade_dist['total'] = grade_dist.sum(axis=1)
grade_dist = grade_dist.sort_values('total', ascending=False)
grade_dist = grade_dist.drop('total', axis=1)

# Define a fancy color palette
colors = px.colors.qualitative.Pastel

# Create stacked bar chart
fig = go.Figure()

# Add bars for each grade level
for i, grade in enumerate(sorted(df['comment_account_grade'].dropna().unique())):
    fig.add_trace(go.Bar(
        name=f'Level {int(grade)}',
        x=grade_dist.index,
        y=grade_dist[grade],
        marker_color=colors[i % len(colors)]  # Cycle through the fancy color palette
    ))

# Update layout
fig.update_layout(
    barmode='stack',
    title='Distribution of User Account Grades by Cuisine Type',
    xaxis_title='Cuisine Type',
    yaxis_title='Number of Reviews',
    height=600,
    xaxis_tickangle=45,
    plot_bgcolor='rgba(240,240,240,0.95)',
    paper_bgcolor='rgba(255,255,255,1)',
    font=dict(size=12),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    margin=dict(t=150)
)

# Add gridlines
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Display the plot
fig.show()

# Print summary statistics
print("\
Grade distribution by cuisine type:")
print("\
Total reviews by cuisine type:")
print(grade_dist.sum(axis=1))


Grade distribution by cuisine type:
Total reviews by cuisine type:
cuisine_category
港式       46416
西式       43766
日本菜      34819
中菜       23371
Other    18076
泰國菜       8161
火鍋        7735
台灣菜       7375
韓國菜       6294
越南菜       3776
咖啡        3673
法國菜       1843
自助餐       1463
素食         832
甜品         210
dtype: int64


In [ ]:
# Get unique restaurants and their dining ways using restaurant_name
unique_restaurants = df.groupby('restaurant_name').agg({
    'cuisine_category': 'first',  # Take the first cuisine category for each restaurant
    'comment_dining_way': 'first',  # Take the first dining way for each restaurant
}).reset_index()

# Create a distribution for dining ways by cuisine type using unique restaurants
dining_dist = pd.crosstab(unique_restaurants['cuisine_category'], unique_restaurants['comment_dining_way'])

# Remove the specified columns
columns_to_remove = ['2天前', '3天前', '4天前', '5天前', '6天前']
dining_dist = dining_dist.drop(columns=columns_to_remove, errors='ignore')

# Calculate percentages
dining_dist_pct = dining_dist.div(dining_dist.sum(axis=1), axis=0) * 100

# Sort by total count in descending order
dining_dist['total'] = dining_dist.sum(axis=1)
dining_dist_pct = dining_dist_pct.loc[dining_dist.sort_values('total', ascending=False).index]

# Define pastel colors
colors = ['#FFB3BA', '#BAFFC9', '#BAE1FF']

# Create stacked bar chart
fig = go.Figure()

# Add bars for each dining way
for i, dining_way in enumerate(dining_dist_pct.columns):
    fig.add_trace(go.Bar(
        name=dining_way,
        x=dining_dist_pct.index,
        y=dining_dist_pct[dining_way],
        marker_color=colors[i % len(colors)]
    ))

# Update layout
fig.update_layout(
    barmode='stack',
    title={
        'text': 'Distribution of Dining Ways by Unique Restaurants in Each Cuisine Type',
        'font': {'size': 24, 'color': '#444444'}
    },
    xaxis_title={
        'text': 'Cuisine Type',
        'font': {'size': 16}
    },
    yaxis_title={
        'text': 'Percentage of Restaurants',
        'font': {'size': 16}
    },
    height=700,
    xaxis_tickangle=45,
    plot_bgcolor='rgba(240,240,240,0.95)',
    paper_bgcolor='rgba(255,255,255,1)',
    font=dict(size=12),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        bgcolor='rgba(255,255,255,0.8)',
        bordercolor='rgba(0,0,0,0.1)',
        borderwidth=1
    ),
    margin=dict(t=150, b=100)
)

# Add gridlines and set y-axis range to 0-100
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(128,128,128,0.2)', range=[0, 100])
fig.update_xaxes(showgrid=False)

# Display the plot
fig.show()

# Print summary statistics
print("\
Dining way distribution percentages by unique restaurants in each cuisine type (sorted by total restaurants):")
print(dining_dist_pct.round(2))

print("\
Total unique restaurants by cuisine type:")
print(dining_dist.sum(axis=1))


Dining way distribution percentages by unique restaurants in each cuisine type (sorted by total restaurants):
comment_dining_way      堂食  外賣自取  外賣送餐
cuisine_category                      
港式                   91.75  7.29  0.97
西式                   95.64  4.18  0.18
日本菜                  97.22  2.55  0.23
中菜                   96.37  3.21  0.42
Other                95.79  3.06  1.15
台灣菜                  88.18  9.55  2.27
泰國菜                  95.90  4.10  0.00
火鍋                   98.91  0.55  0.55
韓國菜                  91.37  7.91  0.72
越南菜                  98.02  1.98  0.00
咖啡                   97.75  2.25  0.00
法國菜                 100.00  0.00  0.00
自助餐                 100.00  0.00  0.00
素食                   95.00  0.00  5.00
甜品                  100.00  0.00  0.00
Total unique restaurants by cuisine type:
cuisine_category
Other    1046
中菜       1434
台灣菜       440
咖啡        178
日本菜      1724
法國菜       108
泰國菜       390
港式       3102
火鍋        366
甜品          8
素食         40
自助餐        74


In [ ]:
# Sort the dining_dist DataFrame by total in descending order
dining_dist = dining_dist.sort_values('total', ascending=False)

# Create stacked bar chart with raw counts
fig = go.Figure()

# Add bars for each dining way
for i, dining_way in enumerate(dining_dist.columns[:-1]):  # Exclude the 'total' column
    fig.add_trace(go.Bar(
        name=dining_way,
        x=dining_dist.index,
        y=dining_dist[dining_way],
        marker_color=colors[i % len(colors)]
    ))

# Update layout
fig.update_layout(
    barmode='stack',
    title={
        'text': 'Distribution of Dining Ways by Restaurants in Each Cuisine Type',
        'font': {'size': 24, 'color': '#444444'}
    },
    xaxis_title={
        'text': 'Cuisine Type',
        'font': {'size': 16}
    },
    yaxis_title={
        'text': 'Number of Unique Restaurants',
        'font': {'size': 16}
    },
    height=700,
    xaxis_tickangle=45,
    plot_bgcolor='rgba(240,240,240,0.95)',
    paper_bgcolor='rgba(255,255,255,1)',
    font=dict(size=12),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        bgcolor='rgba(255,255,255,0.8)',
        bordercolor='rgba(0,0,0,0.1)',
        borderwidth=1
    ),
    margin=dict(t=150, b=100)
)

# Add gridlines
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(128,128,128,0.2)')
fig.update_xaxes(showgrid=False)

# Display the plot
fig.show()

# Print summary statistics
print("\
Dining way distribution by unique restaurants in each cuisine type (raw counts, sorted by total):")
print(dining_dist)


Dining way distribution by unique restaurants in each cuisine type (raw counts, sorted by total):
comment_dining_way    堂食  外賣自取  外賣送餐  total
cuisine_category                           
港式                  1423   113    15   1551
西式                  1053    46     2   1101
日本菜                  838    22     2    862
中菜                   691    23     3    717
Other                501    16     6    523
台灣菜                  194    21     5    220
泰國菜                  187     8     0    195
火鍋                   181     1     1    183
韓國菜                  127    11     1    139
越南菜                   99     2     0    101
咖啡                    87     2     0     89
法國菜                   54     0     0     54
自助餐                   37     0     0     37
素食                    19     0     1     20
甜品                     4     0     0      4
